In [ ]:
import pandas as pd
from IPython.display import display, Markdown

# Load the datasets to examine the content and structure
file_paths = {
    "geo_data_0": "https://practicum-content.s3.us-west-1.amazonaws.com/datasets/geo_data_0.csv",
    "geo_data_1": "https://practicum-content.s3.us-west-1.amazonaws.com/datasets/geo_data_1.csv",
    "geo_data_2": "https://practicum-content.s3.us-west-1.amazonaws.com/datasets/geo_data_2.csv"
}

# Read the files
data_0 = pd.read_csv(file_paths["geo_data_0"])
data_1 = pd.read_csv(file_paths["geo_data_1"])
data_2 = pd.read_csv(file_paths["geo_data_2"])

# Display the first few rows of each dataset to understand their structure
data_0.head(), data_1.head(), data_2.head()

(      id        f0        f1        f2     product
 0  txEyH  0.705745 -0.497823  1.221170  105.280062
 1  2acmU  1.334711 -0.340164  4.365080   73.037750
 2  409Wp  1.022732  0.151990  1.419926   85.265647
 3  iJLyR -0.032172  0.139033  2.978566  168.620776
 4  Xdl7t  1.988431  0.155413  4.751769  154.036647,
       id         f0         f1        f2     product
 0  kBEdx -15.001348  -8.276000 -0.005876    3.179103
 1  62mP7  14.272088  -3.475083  0.999183   26.953261
 2  vyE1P   6.263187  -5.948386  5.001160  134.766305
 3  KcrkZ -13.081196 -11.506057  4.999415  137.945408
 4  AHL4O  12.702195  -8.147433  5.004363  134.766305,
       id        f0        f1        f2     product
 0  fwXo0 -1.146987  0.963328 -0.828965   27.758673
 1  WJtFt  0.262778  0.269839 -2.530187   56.069697
 2  ovLUW  0.194587  0.289035 -5.586433   62.871910
 3  q6cA6  2.236060 -0.553760  0.930038  114.572842
 4  WPMUX -0.515993  1.716266  5.899011  149.600746)

Cada conjunto de datos contiene las siguientes columnas

- `id`: Identificador único para cada pozo petrolífero.
- `f0`, `f1`, `f2`: Rasgos que describen las características del pozo (su significado específico no es esencial).
- `producto`: El volumen de reservas del pozo, medido en miles de barriles.

Ahora que entendemos la estructura de los datos, el siguiente paso es

1. Entrenar un modelo de regresión lineal para cada región utilizando `f0`, `f1` y `f2` como características para predecir el `producto`.
2. Seleccionar los 200 mejores pozos en función de las reservas previstas para cada región.
3. Calcule el beneficio esperado utilizando los detalles de costes e ingresos proporcionados.
4. Realice un análisis de riesgo utilizando bootstrapping para determinar qué regiones cumplen el umbral de riesgo del 2,5%.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

# Prepare a function to train the linear regression model and predict product for each region
def train_and_predict(data):
    X = data[['f0', 'f1', 'f2']]
    y = data['product']

    # Split the data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

    # Initialize and train the model
    model = LinearRegression()
    model.fit(X_train, y_train)

    # Predict on the test set
    y_pred = model.predict(X_test)

    predictions = (pd.DataFrame({'actual': y_train, 'predicted': y_pred})
                    .to_csv('predictions_geo_data_0.csv', index=False))

    # Calculate the RMSE to evaluate the model
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))

    return model, y_pred, y_test, rmse

# Train models and get predictions for each region
model_0, y_pred_0, y_test_0, rmse_0 = train_and_predict(data_0)
model_1, y_pred_1, y_test_1, rmse_1 = train_and_predict(data_1)
model_2, y_pred_2, y_test_2, rmse_2 = train_and_predict(data_2)

rmse_0, rmse_1, rmse_2

(37.756600350261685, 0.890280100102884, 40.14587231134218)

Los valores del error cuadrático medio (RMSE) de los modelos de regresión lineal en los datos de prueba de cada región son los siguientes:

- **Región 0**: RMSE = 37.76
- **Región 1**: RMSE = 0.89
- **Región 2**: RMSE = 40.15

Estos valores indican lo bien que el modelo predice el «producto» (reservas de petróleo) de cada región. Cuanto menor es el RMSE, mejores son las predicciones del modelo, siendo la Región 1 la que presenta el mejor rendimiento de predicción.

A continuación, procederé a seleccionar los 200 mejores pozos en función de las reservas pronosticadas y calcularé las estimaciones de beneficios de cada región.

In [ ]:
# Prepare a function to calculate the profit based on predicted top 200 wells
def calculate_profit(model, data):
    # Predict on the entire dataset
    X = data[['f0', 'f1', 'f2']]
    predictions = model.predict(X)

    # Add predictions to the dataframe
    data['predicted_product'] = predictions

    # Sort by predicted product to get the top 200 wells
    top_wells = data.nlargest(200, 'predicted_product')

    # Total profit calculation
    total_product = top_wells['predicted_product'].sum()  # in thousands of barrels
    profit_per_unit = 4500  # 4500 USD per thousand barrels
    budget = 100_000_000  # 100 million USD

    revenue = total_product * profit_per_unit
    profit = revenue - budget

    return profit

# Calculate the profit for each region
profit_0 = calculate_profit(model_0, data_0.copy())
profit_1 = calculate_profit(model_1, data_1.copy())
profit_2 = calculate_profit(model_2, data_2.copy())

profit_0, profit_1, profit_2


(47012558.575654775, 25244984.21900387, 40933458.03536934)

Los beneficios estimados para cada región, basados en los 200 pozos más importantes, son los siguientes:

- **Región 0**: $47,012,558

- **Región 1**: $25,244,984

- **Región 2**: $40,933,458

La Región 0 presenta el beneficio estimado más elevado, seguida de la Región 2. Sin embargo, tenemos que realizar un análisis de riesgos utilizando bootstrapping para evaluar los riesgos y determinar si el potencial de beneficios es coherente antes de seleccionar la mejor región. Procederé a ese análisis.

In [ ]:
# Perform bootstrapping with reduced samples for performance
def bootstrap_profit_optimized(model, data, n_samples=100):
    profits = []
    np.random.seed(42)  # for reproducibility

    for _ in range(n_samples):
        # Sample the dataset with replacement
        sampled_data = data.sample(frac=1, replace=True)

        # Calculate profit for the sampled data
        profit = calculate_profit(model, sampled_data)
        profits.append(profit)

    # Calculate the 2.5th percentile for risk assessment (threshold for losses)
    risk_threshold = np.percentile(profits, 2.5)
    average_profit = np.mean(profits)
    loss_probability = (np.array(profits) < 0).mean() * 100  # percentage of cases with losses

    return average_profit, risk_threshold, loss_probability

# Retry with reduced number of bootstrapping samples for each region
avg_profit_0, risk_0, loss_prob_0 = bootstrap_profit_optimized(model_0, data_0.copy())
avg_profit_1, risk_1, loss_prob_1 = bootstrap_profit_optimized(model_1, data_1.copy())
avg_profit_2, risk_2, loss_prob_2 = bootstrap_profit_optimized(model_2, data_2.copy())

display(Markdown('**Region 0** Average Profit'))
display(avg_profit_0, risk_0, loss_prob_0)
display(Markdown('**Region 1** Average Profit'))
display(avg_profit_1, risk_1, loss_prob_1)
display(Markdown('**Region 2** Average Profit'))
display(avg_profit_2, risk_2, loss_prob_2)

**Region 0** Average Profit

46976620.067325935

46010460.216132544

0.0

**Region 1** Average Profit

...

0.0

**Region 2** Average Profit

40946148.77455929

39964315.06935108

0.0

Los resultados del análisis bootstrapping muestran los siguientes beneficios medios, umbrales de riesgo y probabilidades de pérdida para cada región:

- **Región 0**:
  - Beneficio medio: 46.976.620 $.

  - Umbral de riesgo (percentil 2,5): $46,010,460

  - Probabilidad de pérdida: 0%.

- **Región 1**:
  - Beneficio medio: 25.245.871 $.

  - Umbral de riesgo (percentil 2,5): $25,207,048

  - Probabilidad de pérdida: 0%.


- **Región 2**:
  - Beneficio medio: $40.946.149
  
  - Umbral de riesgo (percentil 2,5): $39,964,315

  - Probabilidad de pérdida: 0%.

Todas las regiones no presentan riesgo de pérdida, ya que la probabilidad de incurrir en pérdidas es del 0% para cada una de ellas. Sin embargo, la **Región 0** tiene el beneficio medio más alto, lo que la convierte en la elección óptima para desarrollar nuevos pozos petrolíferos basándose tanto en el potencial de beneficio como en el análisis de riesgo.